In [1]:
from sepal_ui import widgetFactory as wf
from sepal_ui import widgetBinding as wb

import ipyvuetify as v
from utils import messages as ms

In [2]:
#use a class to define your input and output in order to have mutable variables
class Mspa_process_IO:
    def __init__(self):
        #set up your inputs
        self.foreground_connectivity = None
        self.edge_width = None
        self.transition_core = True
        self.separate_feature = True
        self.statistics = True
        
        #set up output
        self.tif_map = None
        
mspa_process_io = Mspa_process_IO()

In [3]:
#create an empty result tile that will be filled with displayable plot, map, links, text
id_ = "mspa_result_widget"
title = "Results"
mspa_results = wf.Tile(id_, title, inputs=['No results to display yet'])

In [4]:
#create the output alert 
mspa_process_output = wf.OutputWidget(ms.MSPA_MESSAGE)

#create the button that start your process
mspa_process_btn = wf.ProcessBtn('Run mspa analysis')

#create the widgets following ipyvuetify requirements 
connectivity = [4, 8] #using text input to transform it into 'confXX'
mspa_connectivity = v.Select(
    items=connectivity, 
    label='Foreground connectivity', 
    v_model=None
)
mspa_edge_slider = v.Slider(
    label= 'Edge width', 
    class_="mt-5", 
    thumb_label='always', 
    v_model=0
)
mspa_transition = v.Switch(label="Transition Core - Loop/Bridge", v_model=True)
mspa_feature = v.Switch(label="Separate internal from external features", v_model=True)
mspa_statistics = v.Switch(label="Compute statistics", v_model=True)


mspa_process_inputs = [
    mspa_connectivity, 
    mspa_edge_slider, 
    mspa_transition, 
    mspa_feature, 
    mspa_statistics
]

#bind the widget to the inputs
wb.bind(mspa_connectivity, mspa_process_io, 'foreground_connectivity', mspa_process_output)
wb.bind(mspa_edge_slider, mspa_process_io, 'edge_width', mspa_process_output)
wb.bind(mspa_transition, mspa_process_io, 'transition_core', mspa_process_output)
wb.bind(mspa_feature, mspa_process_io, 'separate_feature', mspa_process_output)
wb.bind(mspa_statistics, mspa_process_io, 'statistics', mspa_process_output)


#create a process tile
id_ = "mspa_process_widget"
title = 'Create fragmentation map'

mspa_process = wf.Tile(
    id_, 
    title, 
    btn=mspa_process_btn, 
    inputs=mspa_process_inputs, 
    output=mspa_process_output
)

In [5]:
#bind the button to the process by writing a custom function
from scripts import run
from sepal_ui.scripts import utils
from functools import partial

def process_start(widget, event, data, output, mspa_results):    
    
    #toggle the loading button
    utils.toggleLoading(widget)
    
    #debug
    asset = 'users/bornToBeAlive/aoi_PU'
    threshold = 20
    clip_map = '/home/prambaud/gfc_wrapper_results/gfc/PU_20_merged_gfc_map.tif'
    
    #load inputs
    #asset = getattr(gfc_aoi_IO, 'assetId')
    #threshold = getattr(gfc_viz_io, 'threshold')
    #clip_map = getattr(gfc_export_io, 'clip_map')
    foreground_connectivity = getattr(mspa_process_io, 'foreground_connectivity')
    edge_width = getattr(mspa_process_io,'edge_width')
    transition_core = getattr(mspa_process_io, 'transition_core')
    separate_feature = getattr(mspa_process_io, 'separate_feature')
    statistics = getattr(mspa_process_io, 'statistics')
    
    #check inputs 
    if not wb.checkInput(asset, output, ms.NO_AOI): return utils.toggleLoading(widget)
    if not wb.checkInput(clip_map, output, ms.NO_MAP): return utils.toggleLoading(widget)
    if not wb.checkInput(foreground_connectivity, output, ms.NO_INPUT): return utils.toggleLoading(widget)
    
    #launch any process you want
    tif_map = run.mspaAnalysis(
        clip_map=clip_map, 
        assetId=asset, 
        threshold=threshold, 
        foreground_connectivity=foreground_connectivity, 
        edge_width=edge_width, 
        transition_core=transition_core, 
        separate_feature=separate_feature, 
        statistics=statistics,
        output=output
    )
    
    setattr(mspa_process_io, 'tif_map', tif_map)
    
    #display the link 
    mspa_results.children[0].children = [
        mspa_results.children[0].children[0],
        wf.DownloadBtn("Download mspa map .tif", tif_map)
    ]
    
    #toggle the loading button
    utils.toggleLoading(widget)

mspa_process_btn.on_event('click', partial(
    process_start, 
    output=mspa_process_output,
    mspa_results=mspa_results
))

In [6]:
#this tiles will only be displayed if you launch voila from this file 
mspa_process

Layout(align_center=True, children=[Card(children=[Html(children=['Create fragmentation map'], tag='h2'), Flex…

bash /home/prambaud/gfc_wrapper_results/tmp/mspa/sepal_mspa


FileNotFoundError: [Errno 2] No such file or directory: '/home/prambaud/gfc_wrapper_results/tmp/mspa/output/input_8_7_1_1_1_stat.txt'

In [7]:
#this tiles will only be displayed if you launch voila from this file 
mspa_results

Layout(align_center=True, children=[Card(children=[Html(children=['Results'], tag='h2'), Flex(children=[Layout…